In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [23]:
hate_dev=pd.read_csv('dev.hate.csv', encoding='UTF-8')
hate_test=pd.read_csv('train.hate.csv', encoding='UTF-8')
hate_dev.replace({'hate': 2, 'none': 0, 'offensive': 1}, inplace=True)
hate_test.replace({'hate': 2, 'none': 0, 'offensive': 1}, inplace=True)

In [24]:
X_train=hate_dev['comments']
X_test=hate_test['comments']
y_train=hate_dev['label']
y_test=hate_test['label']
X_train.head(4)


0                   송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.
1                                       지현우 나쁜놈
2    알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라
3                              설마 ㅈ 현정 작가 아니지??
Name: comments, dtype: object

In [25]:
from transformers import BertTokenizer, TFBertForSequenceClassification
 
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
 
def encode(data, tokenizer):
    input_ids = []
    attention_masks = []
    token_type_ids = []
    for text in data:
        tokenized_text = tokenizer.encode_plus(text,
                                            max_length=50,
                                            add_special_tokens = True,
                                            pad_to_max_length=True,
                                            return_attention_mask=True,
                                              truncation=True)
        input_ids.append(tokenized_text['input_ids'])
        attention_masks.append(tokenized_text['attention_mask'])
        token_type_ids.append(tokenized_text['token_type_ids'])
    
    return input_ids, attention_masks, token_type_ids

In [26]:
#학습 데이터
train_input_ids, train_attention_masks, train_token_type_ids = encode(X_train, tokenizer)
 
#테스트 데이터
test_input_ids, test_attention_masks, test_token_type_ids = encode(X_test, tokenizer)



In [27]:
print(type(train_input_ids[0][0]))

<class 'int'>


In [28]:
def map_example_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
      }, label
      
      
def data_encode(input_ids_list, attention_mask_list, token_type_ids_list, label_list):
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(map_example_to_dict)
    
BATCH_SIZE=32
    
train_data_encoded = data_encode(train_input_ids, train_attention_masks, train_token_type_ids,y_train).shuffle(10000).batch(BATCH_SIZE)
test_data_encoded = data_encode(test_input_ids, test_attention_masks, test_token_type_ids, y_test).batch(BATCH_SIZE)

In [29]:
model = TFBertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3
)
 
 
optimizer = tf.keras.optimizers.Adam(1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
 
NUM_EPOCHS = 5
history = model.fit(train_data_encoded, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=test_data_encoded)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


15/15 [==============================] - ETA: 0s - loss: 1.0926 - accuracy: 0.3949 

KeyboardInterrupt: ignored

In [30]:
# loss 확인 
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','val'])
plt.show()
 
 
# accuracy 확인 
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','val'])
plt.show()

NameError: ignored